In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/ColabNotebooks/')

In [3]:
%pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 9.6 MB/s eta 0:00:00


In [4]:
import torch

from torchmetrics.classification import BinaryF1Score, BinaryAccuracy, BinarySpecificity, BinaryPrecision, BinaryRecall
from model.wide_res_net import WideResNet
from model.smooth_cross_entropy import smooth_crossentropy
from coronary import Coronary
from utility.log import Log
from utility.initialize import initialize
from utility.step_lr import StepLR
import torch.nn as nn


In [5]:

def train(model, device, train_loader, optimizer, log, scheduler):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in train_loader:
        inputs, targets = (b.to(device) for b in batch)
        inputs = inputs.permute(0, 3, 2, 1)

        optimizer.zero_grad()

        predictions = model(inputs)
        loss = smooth_crossentropy(predictions, targets, smoothing=0.001)
        loss.mean().backward()

        optimizer.step()

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            scheduler(epoch)

In [6]:
def test(model, device, test_loader, log):
    model.eval()
    log.eval(len_dataset=len(test_loader))

    with torch.no_grad():
        for batch in test_loader:
            inputs, targets = (b.to(device) for b in batch)
            inputs = inputs.permute(0, 3, 2, 1)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets, smoothing=0.001)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())

In [7]:
class ModifiedViT(nn.Module):
    def __init__(self, pretrained_model):
        super(ModifiedViT, self).__init__()
        self.pretrained = pretrained_model
        self.pretrained.head = nn.Identity()  # remove the existing linear layer
        self.new_head = nn.Sequential(
            nn.Linear(1000, 2),  # Adjust the input size to match the output size of the ViT model
        )

    def forward(self, x):
        x = self.pretrained(x)
        x = self.new_head(x)
        return x


In [8]:
from torchvision import models

if __name__ == "__main__":

    num_epochs = 50
    model_name_path = '/content/drive/MyDrive/ColabNotebooks/results_logs/'
    model_name = 'vit-sgd-224'
    model_name = model_name_path + model_name
    initialize(seed=42)
    log = Log(log_each=1, log_name=model_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    vit_b_16 = models.vit_b_16(pretrained=True)
    model = ModifiedViT(vit_b_16).to(device)
    model.eval()

    dataset = Coronary(batch_size=100, threads=2, img_size_definition=224)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.1)
    scheduler = StepLR(optimizer, 0.0001, num_epochs)

    for epoch in range(num_epochs):
        train(model, device, dataset.train, optimizer, log, scheduler)
        test(model, device, dataset.valid, log)

    log.flush()
    # ┃          49  ┃      0.1876  │      0.86 %  ┃   4.000e-05  │       00:02  ┃      0.3099  │     74.00 %  ┃

    # torch.save(model.state_dict(), '../model_trained/'+model_name+'.pth')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:00<00:00, 396MB/s]


┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨


/content/drive/MyDrive/ColabNotebooks/coronary.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  train_tensor_x = torch.Tensor(train_xs)


┃           0  ┃      0.5235  │      0.73 %  ┃   1.000e-04  │       00:16  ┃      0.4462  │     91.00 %  ┃
┃           1  ┃      0.4704  │      0.81 %  ┃   1.000e-04  │       00:08  ┃      0.4456  │     90.33 %  ┃
┃           2  ┃      0.4582  │      0.83 %  ┃   1.000e-04  │       00:08  ┃      0.4437  │     88.00 %  ┃
┃           3  ┃      0.4474  │      0.84 %  ┃   1.000e-04  │       00:08  ┃      0.4397  │     88.00 %  ┃
┃           4  ┃      0.4369  │      0.85 %  ┃   1.000e-04  │       00:08  ┃      0.4356  │     88.67 %  ┃
┃           5  ┃      0.4266  │      0.85 %  ┃   1.000e-04  │       00:08  ┃      0.4314  │     88.67 %  ┃
┃           6  ┃      0.4166  │      0.86 %  ┃   1.000e-04  │       00:08  ┃      0.4273  │     88.67 %  ┃
┃           7  ┃      0.4070  │      0.86 %  ┃   1.000e-04  │       00:08  ┃      0.4232  │     88.67 %  ┃
┃           8  ┃      0.3974  │      0.87 %  ┃   1.000e-04  │       00:08  ┃      0.4181  │     88.67 %  ┃
┃           9  ┃      0.3882  │      

In [9]:
metricF1 = BinaryF1Score().to(device)
metricAcc = BinaryAccuracy().to(device)
metricSpecificity = BinarySpecificity().to(device)
metricPrecision = BinaryPrecision().to(device)
metricRecall = BinaryRecall().to(device)

In [10]:
inputs = dataset.test_x.permute(0, 3, 2, 1).to(device)
predictions = model(inputs)
f1 = metricF1(torch.argmax(predictions, 1), dataset.test_y.to(device))
acc = metricAcc(torch.argmax(predictions, 1), dataset.test_y.to(device))
specificity = metricSpecificity(torch.argmax(predictions, 1), dataset.test_y.to(device))
precision = metricPrecision(torch.argmax(predictions, 1), dataset.test_y.to(device))
recall = metricRecall(torch.argmax(predictions, 1), dataset.test_y.to(device))

with open(model_name+'.txt', 'a') as file:
                file.write(f"\n \n F1: {f1}\n Accuracy: {acc}\n Specificity:{specificity}\n Precision: {precision}\n Recall: {recall}")

print(f1)
print(acc)
print(precision)
print(specificity)
print(recall)


tensor(0.9505, device='cuda:0')
tensor(0.9500, device='cuda:0')
tensor(0.9412, device='cuda:0')
tensor(0.9400, device='cuda:0')
tensor(0.9600, device='cuda:0')
